In [4]:
import json
from utils import generate_text, AIRequestError

def refine_scene(scene_json_path: str, scene_id: int, api_key: str,
                 model: str = "gemini-2.5-flash", out_path: str = "refined_prompt.txt") -> str:
    """
    Refine one scene in the script to improve SSML voiceover and visual description.
    - Input: path to script.json and scene_id to refine
    - Output: refined Veo 3 prompt (saved to a text file)
    """

    # --- Load script ---
    with open(scene_json_path, "r", encoding="utf-8") as f:
        scenes = json.load(f)

    scene = next((s for s in scenes if s["scene_id"] == scene_id), None)
    if not scene:
        raise ValueError(f"Scene {scene_id} not found in {scene_json_path}")

    ssml = scene["ssml"]
    visual_desc = scene["visual_desc"]
    transition_hint = scene.get("transition_hint", "")

    # --- Prompt để refine ---
    refine_prompt = f"""
You are a professional Vietnamese TV news editor and an English-language visual director.

Your task:
1. **Enhance the SSML** to sound more natural, emotional, and professional for a Vietnamese national news anchor.
   - Add appropriate <break>, <emphasis>, <prosody> tags.
   - Keep the meaning unchanged, but improve rhythm and stress.
   - Output must remain valid SSML in Vietnamese (<speak>...</speak>).

2. **Translate and expand the visual description** into detailed English for Veo 3.
   - Describe camera movement, lighting, mood, composition, and colors.
   - Make it cinematic but realistic for a national news broadcast.
   - Include any transition hint for continuity: {transition_hint}
   - Force the audio to follow exaclty the SSML provided.
   

3. **Output in JSON** with this structure:
{{
  "scene_id": {scene_id},
  "refined_ssml": "<speak>...</speak>",
  "veo_prompt_en": "..."
}}

Input data:
- Original SSML:
{ssml}

- Original visual description:
{visual_desc}
    """

    try:
        result = generate_text(refine_prompt, api_key=api_key, model=model, max_tokens=4000)
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(result)
        print(f"✅ Refined prompt saved to {out_path}")
        return result
    except AIRequestError as e:
        print(f"❌ API error while refining scene: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


In [ ]:
result = refine_scene(r"D:\Study2\thucchien-video-generation\outputs\scripts\script.json", scene_id=1)

✅ Refined prompt saved to refined_prompt.txt
